# LAB7 Assignment
> 11912911 张子越
------

## LAB Assignment
### Exercise 1 logistic regression (20 points )
This exercise uses dataset digit01.csv , which has 13 columns, and the last column is the dependent variable. 

This part requires you to implement a `logistic regression` using the pytorch framework (defining a logistic regression class that inherits `nn.module`). To test your model, we provide a dataset `digit01.csv` which is in the **datasets folder**. This dataset requires you to divide the training set and the test set by yourself, and it is recommended that 80% of the training set and 20% of the test set be used.

+ load datasets

In [1]:
########### Write Your Code Here ###########
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore") 

df = pd.read_csv("./datasets/digit01.csv", header=None)

print(df.shape)
df.head()
############################################

FileNotFoundError: [Errno 2] No such file or directory: './datasets/digit01.csv'

In [2]:
# col 13 is y
print(df[12].value_counts())
y = df[12]
X = df.drop(12, axis=1)
X.shape, y.shape

0    32
1    32
Name: 12, dtype: int64


((64, 12), (64,))

+ Splitting dataset into 80% Training and 20% Testing Data:

In [3]:
########### Write Your Code Here ###########
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
############################################

((51, 12), (51,), (13, 12), (13,))

In [4]:
# change type to tensor
import torch 

X_train = torch.from_numpy(X_train.values).float()
y_train = torch.from_numpy(y_train.values).float().unsqueeze(1)
X_test  = torch.from_numpy(X_test .values).float()
y_test  = torch.from_numpy(y_test .values).float().unsqueeze(1)

+ Define a LogisticRegression subclass of nn. Module

In [5]:
# Define a LogisticRegression subclass of nn.Module.
########### Write Your Code Here ###########
import torch
from torch import nn
from torch.nn import functional as f

class MyLogisticRegression(nn.Module):
    def __init__(self, in_features, out_features) -> None:
        super(MyLogisticRegression, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
        assert(in_features > 0 and out_features > 0)

        self.device = torch.device('cpu')
        self.dtype = torch.float

        self.flattern = nn.Flatten()
        self.linear_sigmoid_stack = nn.Sequential(
            nn.Linear(self.in_features, self.out_features, device=self.device, dtype=self.dtype),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.flattern(x)
        y_pred = self.linear_sigmoid_stack(x)
        return y_pred


############################################

+ Create the model

In [6]:
########### Write Your Code Here ###########

m, n = X_train.shape[1], 1
model = MyLogisticRegression(m, n)

############################################

 + Loss function

In [7]:
########### Write Your Code Here ###########

# criterions = torch.nn.MSELoss(reduction='sum')
criterions = torch.nn.BCELoss(size_average=True)

############################################

+ The optimizer

In [8]:
########### Write Your Code Here ###########

# optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001)

############################################

+ training Model

In [9]:
########### Write Your Code Here ###########

for epoch in range(2000):
    model.train()
    optimizer.zero_grad()
    # Forward
    y_pred = model(X_train)
    loss = criterions(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

############################################

epoch: 100, loss = 0.7086
epoch: 200, loss = 0.6852
epoch: 300, loss = 0.6631
epoch: 400, loss = 0.6421
epoch: 500, loss = 0.6223
epoch: 600, loss = 0.6034
epoch: 700, loss = 0.5856
epoch: 800, loss = 0.5686
epoch: 900, loss = 0.5524
epoch: 1000, loss = 0.5370
epoch: 1100, loss = 0.5224
epoch: 1200, loss = 0.5084
epoch: 1300, loss = 0.4951
epoch: 1400, loss = 0.4824
epoch: 1500, loss = 0.4703
epoch: 1600, loss = 0.4587
epoch: 1700, loss = 0.4476
epoch: 1800, loss = 0.4370
epoch: 1900, loss = 0.4269
epoch: 2000, loss = 0.4172



+ Model Performance


In [10]:
########### Write Your Code Here ###########

with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.squeeze().numpy()
    print(y_pred)
    for i in range(len(y_pred)):
        if y_pred[i] > 0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    print(y_pred)
    y_test_numpy = y_test.squeeze().numpy()
    print(y_test_numpy)
    acc = len(y_pred[y_pred == y_test_numpy]) / float(X_test.shape[0])
    print(f'accuracy: {acc:.4f}')

############################################

[0.65100086 0.25052527 0.45733142 0.582207   0.2786178  0.64463115
 0.5874424  0.627756   0.27040705 0.16217469 0.2509038  0.22235188
 0.5703614 ]
[1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1.]
[1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1.]
accuracy: 0.9231


In [11]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_numpy, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92         6
         1.0       1.00      0.86      0.92         7

    accuracy                           0.92        13
   macro avg       0.93      0.93      0.92        13
weighted avg       0.93      0.92      0.92        13



### Exercise 2  Handwriting recognition with MLP

Like last week's lab , your task in this section is also about recognizing handwritten digits, but you are required to use MLP to complete the exercise. It is recommended that you define an MLP class, which is a subclass of `nn.module`.

<font color='red' size=4>Note that your accuracy in this section will directly determine your score.</font>

For this exercise we use the `minist` dataset.

+ load datasets

In [2]:
########### Write Your Code Here ###########

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

import warnings
warnings.filterwarnings("ignore") 

train_ds = datasets.MNIST('./datasets', train=True, transform=transforms.ToTensor(), download=True)

test_ds = datasets.MNIST('./datasets/', train=False, transform=transforms.ToTensor(), download=True)

print(train_ds.data.shape, train_ds.targets.shape, test_ds.data.shape, test_ds.targets.shape)

X_train = train_ds.data.float().flatten(1)
y_train = train_ds.targets.long()
X_test  = test_ds .data.float().flatten(1)
y_test  = test_ds .targets.long()

print(X_train.dtype, y_train.dtype)

# X_train_ = torch.flatten(X_train, 1)

print(X_train.shape, y_train.unique())

X_train[:5], y_train[:5]
############################################

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw

torch.Size([60000, 28, 28]) torch.Size([60000]) torch.Size([10000, 28, 28]) torch.Size([10000])
torch.float32 torch.int64
torch.Size([60000, 784]) tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([5, 0, 4, 1, 9]))

In [3]:
from torch.nn.functional import normalize
print(X_train.std())
print(X_train[0].sum())

# X_train_norm = normalize(X_train)
# print(X_train_norm.std())
# print(X_train_norm[0].sum())

# X_train = X_train_norm
# X_test = normalize(X_test)

tensor(78.5675)
tensor(27525.)


+ Define a MLP subclass of nn. Module

In [4]:
########### Write Your Code Here ###########

torch.manual_seed(123)

class MyMLP(nn.Module):
    def __init__(self, ni, h1, h2, no) -> None:
        super(MyMLP, self).__init__()
        self.MLP_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(ni, h1),
            # nn.BatchNorm1d(h1),
            nn.ReLU(),
            nn.Dropout(0.5),
            # nn.Linear(h1, h2),
            # # nn.BatchNorm1d(h2),
            # nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(h1, no)
        )

    def forward(self, x):
        logits = self.MLP_stack(x)
        return logits

############################################

+ Create the model

In [5]:
########### Write Your Code Here ###########

i, h1, h2, o = 784, 512, 128, 10

model = MyMLP(i, 392, h2, o)

############################################

 + Loss function

In [6]:
########### Write Your Code Here ###########

criterion = nn.CrossEntropyLoss()

############################################

+ The optimizer

In [7]:
########### Write Your Code Here ###########

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

############################################

+ training Model

In [8]:
########### Write Your Code Here ###########

for epoch in range(2000):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    y_pred  = model(X_train)
    # Compute Loss
    loss = criterion(y_pred, y_train)
    # Backward pass
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

# 120: 0.4216
############################################

epoch: 100, loss = 1.1071
epoch: 200, loss = 0.7333
epoch: 300, loss = 0.5919
epoch: 400, loss = 0.5089
epoch: 500, loss = 0.4586
epoch: 600, loss = 0.4308
epoch: 700, loss = 0.3826
epoch: 800, loss = 0.3657
epoch: 900, loss = 0.3431
epoch: 1000, loss = 0.3354
epoch: 1100, loss = 0.3141
epoch: 1200, loss = 0.3026
epoch: 1300, loss = 0.2908
epoch: 1400, loss = 0.2806
epoch: 1500, loss = 0.2788
epoch: 1600, loss = 0.2652
epoch: 1700, loss = 0.2548
epoch: 1800, loss = 0.2516
epoch: 1900, loss = 0.2391
epoch: 2000, loss = 0.2342


+ Model Performance

In [19]:
########### Write Your Code Here ###########

with torch.no_grad():
    model.eval()
    logits = model(X_test)
    y_pred = nn.Softmax(dim=1)(logits)
    y_predicted_cls = y_pred.argmax(1)
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

# 0.9154
# 0.9572
############################################

accuracy: 0.9572


In [20]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.95      0.96      0.96      1032
           3       0.96      0.96      0.96      1010
           4       0.96      0.96      0.96       982
           5       0.95      0.95      0.95       892
           6       0.97      0.96      0.97       958
           7       0.96      0.94      0.95      1028
           8       0.93      0.94      0.93       974
           9       0.94      0.93      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



### Exercise 3  Questions (10 points )
1.What's the difference between logistic regression and Perceptron?

Perception is nonlinear while logistic regression is linear. MLP uses CrossEntropy as its loss function. 


2.Advantages and disadvantages of neural networks?

Advantages: 1. There are more abundant data features. 2. It can fit nonlinear data.

Disadvantages: 1. it is very complex. 2. It is hard to adjust the structures and parameters. 3. Gradient explosion & gradient disappearance. 4. Often cost more compute resources. 

3.What is the role of Activation Function in Neural networks?

Limit the range of original data, change the original relationship. Change the linear relationship to fit the nonlinear mapping of the model. 
